In [0]:
%%sh
# Download the CMS Open Payments ZIP file
curl -L https://download.cms.gov/openpayments/PGYR2023_P01302025_01212025.zip -o /tmp/openpayments_2023.zip

# Unzip the file to a directory
unzip /tmp/openpayments_2023.zip -d /tmp/openpayments_2023

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  752M  100  752M    0     0   185M      0  0:00:04  0:00:04 --:--:--  185M


Archive:  /tmp/openpayments_2023.zip
  inflating: /tmp/openpayments_2023/OP_PGYR2023_README_P01302025.txt  
  inflating: /tmp/openpayments_2023/OP_DTL_OWNRSHP_PGYR2023_P01302025_01212025.csv  
  inflating: /tmp/openpayments_2023/OP_DTL_GNRL_PGYR2023_P01302025_01212025.csv  
  inflating: /tmp/openpayments_2023/OP_DTL_RSRCH_PGYR2023_P01302025_01212025.csv  
  inflating: /tmp/openpayments_2023/OP_REMOVED_DELETED_PGYR2023_P01302025_01212025.csv  


In [0]:
# Reading the CSV file into a Spark DataFrame
df_1 = spark.read.csv("file:/tmp/openpayments_2023/OP_DTL_GNRL_PGYR2023_P01302025_01212025.csv", header=True, inferSchema=True)

# Saving it as a Delta table
df_1.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("General_2023_OpenPayments")

In [0]:
df_1.columns

Out[7]: ['Change_Type',
 'Covered_Recipient_Type',
 'Teaching_Hospital_CCN',
 'Teaching_Hospital_ID',
 'Teaching_Hospital_Name',
 'Covered_Recipient_Profile_ID',
 'Covered_Recipient_NPI',
 'Covered_Recipient_First_Name',
 'Covered_Recipient_Middle_Name',
 'Covered_Recipient_Last_Name',
 'Covered_Recipient_Name_Suffix',
 'Recipient_Primary_Business_Street_Address_Line1',
 'Recipient_Primary_Business_Street_Address_Line2',
 'Recipient_City',
 'Recipient_State',
 'Recipient_Zip_Code',
 'Recipient_Country',
 'Recipient_Province',
 'Recipient_Postal_Code',
 'Covered_Recipient_Primary_Type_1',
 'Covered_Recipient_Primary_Type_2',
 'Covered_Recipient_Primary_Type_3',
 'Covered_Recipient_Primary_Type_4',
 'Covered_Recipient_Primary_Type_5',
 'Covered_Recipient_Primary_Type_6',
 'Covered_Recipient_Specialty_1',
 'Covered_Recipient_Specialty_2',
 'Covered_Recipient_Specialty_3',
 'Covered_Recipient_Specialty_4',
 'Covered_Recipient_Specialty_5',
 'Covered_Recipient_Specialty_6',
 'Covered_Recipi

In [0]:
# File location and type
file_location = "/FileStore/tables/OP_CVRD_RCPNT_PRFL_SPLMTL_P01302025_01212025.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df_2 = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

df_2.columns

Out[8]: ['Covered_Recipient_Profile_Type',
 'Covered_Recipient_Profile_ID',
 'Associated_Covered_Recipient_Profile_ID_1',
 'Associated_Covered_Recipient_Profile_ID_2',
 'Covered_Recipient_NPI',
 'Covered_Recipient_Profile_First_Name',
 'Covered_Recipient_Profile_Middle_Name',
 'Covered_Recipient_Profile_Last_Name',
 'Covered_Recipient_Profile_Suffix',
 'Covered_Recipient_Profile_Alternate_First_Name',
 'Covered_Recipient_Profile_Alternate_Middle_Name',
 'Covered_Recipient_Profile_Alternate_Last_Name',
 'Covered_Recipient_Profile_Alternate_Suffix',
 'Covered_Recipient_Profile_Address_Line_1',
 'Covered_Recipient_Profile_Address_Line_2',
 'Covered_Recipient_Profile_City',
 'Covered_Recipient_Profile_State',
 'Covered_Recipient_Profile_Zipcode',
 'Covered_Recipient_Profile_Country_Name',
 'Covered_Recipient_Profile_Province_Name',
 'Covered_Recipient_Profile_Primary_Specialty',
 'Covered_Recipient_Profile_OPS_Taxonomy_1',
 'Covered_Recipient_Profile_OPS_Taxonomy_2',
 'Covered_Recipient_Pr

In [0]:
from pyspark.sql.functions import col, desc, count, sum, concat_ws

In [0]:
analysis1 = df_1.filter(
    (col("Nature_of_Payment_or_Transfer_of_Value") == "Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program") & 
    (col("Total_Amount_of_Payment_USDollars") > 1000)
).groupBy("Nature_of_Payment_or_Transfer_of_Value") \
 .agg(count("*").alias("count"), sum("Total_Amount_of_Payment_USDollars").alias("total_amount")) \
 .orderBy(desc("count"))
analysis1.show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------+------+--------------+
|Nature_of_Payment_or_Transfer_of_Value                                                                                                            |count |total_amount  |
+--------------------------------------------------------------------------------------------------------------------------------------------------+------+--------------+
|Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program|164092|5.5829497072E8|
+--------------------------------------------------------------------------------------------------------------------------------------------------+------+--------------+



The nature of payments with reimbursement amounts above $1,000 shows "Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program" as the most prevalent type with 164,092 occurrences worth approximately $558.3 million which represents substantial financial investment in non-consulting professional services in healthcare.

In [0]:
analysis2 = df_1.groupBy("Nature_of_Payment_or_Transfer_of_Value") \
    .agg(count("*").alias("count")) \
    .orderBy(desc("count")) \
    .limit(10)

analysis2.show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|Nature_of_Payment_or_Transfer_of_Value                                                                                                            |count   |
+--------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|Food and Beverage                                                                                                                                 |13378464|
|Travel and Lodging                                                                                                                                |545086  |
|Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program|236628  |
|Consulting Fee                                     

The "Food and Beverage" category stands as the most frequently recorded payment type across 13.3 million instances while "Travel and Lodging" remains distant with 545,086 occurrences which indicates most industry-physician transactions involve modest hospitality payments rather than direct compensation.

In [0]:
analysis3 = df_1.groupBy("Nature_of_Payment_or_Transfer_of_Value") \
    .agg(sum("Total_Amount_of_Payment_USDollars").alias("total_amount")) \
    .orderBy(desc("total_amount")) \
    .limit(10)

analysis3.show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|Nature_of_Payment_or_Transfer_of_Value                                                                                                            |total_amount        |
+--------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|Royalty or License                                                                                                                                |1.19217456302E9     |
|Compensation for services other than consulting, including serving as faculty or as a speaker at a venue other than a continuing education program|5.946326876500002E8 |
|Consulting Fee                                                                                                                                    |5.

The payments classified as "Royalty or License" reach approximately $1.19 billion while "Compensation for services other than consulting" amounts to $594.6 million and "Consulting Fee" totals $514.9 million as the most significant industry-physician financial transactions based on their total value.

In [0]:
joined_df = df_1.join(
    df_2,
    df_1["Covered_Recipient_Profile_ID"] == df_2["Covered_Recipient_Profile_ID"],
    "inner"
)

analysis4 = joined_df.groupBy("Covered_Recipient_Profile_Primary_Specialty") \
    .agg(sum("Total_Amount_of_Payment_USDollars").alias("total_amount")) \
    .orderBy(desc("total_amount")) \
    .limit(10)

analysis4.show(truncate=False)

+------------------------------------------------------------------------------------------------+-------------------+
|Covered_Recipient_Profile_Primary_Specialty                                                     |total_amount       |
+------------------------------------------------------------------------------------------------+-------------------+
|Allopathic & Osteopathic Physicians|Orthopaedic Surgery                                         |3.777555961599989E8|
|Allopathic & Osteopathic Physicians|Psychiatry & Neurology|Neurology                            |8.680715036999995E7|
|Allopathic & Osteopathic Physicians|Neurological Surgery                                        |8.587046902999997E7|
|Allopathic & Osteopathic Physicians|Dermatology                                                 |8.076589144000061E7|
|null                                                                                            |7.78133707000005E7 |
|Allopathic & Osteopathic Physicians|Orthopaedic

Specialists in Orthopaedic Surgery received the largest total payments ($377.8 million) while Neurology and Neurological Surgery specialists received $86.8 million and $85.9 million respectively which demonstrates industry financial relationships primarily focus on surgical and neurological fields that probably involve medical devices and specialized pharmaceuticals and procedures.

In [0]:
physician_totals = df_1 \
    .filter(col("Covered_Recipient_Type") == "Covered Recipient Physician") \
    .groupBy("Covered_Recipient_Profile_ID") \
    .agg(sum("Total_Amount_of_Payment_USDollars").alias("total_payments"))

# Join with the physician profile data to get the names
physician_totals_with_names = physician_totals.join(
    df_2.select(
        "Covered_Recipient_Profile_ID", 
        "Covered_Recipient_Profile_First_Name", 
        "Covered_Recipient_Profile_Last_Name",
        "Covered_Recipient_Profile_Primary_Specialty"
    ),
    physician_totals["Covered_Recipient_Profile_ID"] == df_2["Covered_Recipient_Profile_ID"],
    "left"
)

# Get the top 10 physicians by total payment amount
top_10_physicians = physician_totals_with_names \
    .orderBy(desc("total_payments")) \
    .limit(10)

# Show the results
top_10_physicians.select(
    "Covered_Recipient_Profile_First_Name",
    "Covered_Recipient_Profile_Last_Name",
    "Covered_Recipient_Profile_Primary_Specialty",
    "total_payments"
).show()

+------------------------------------+-----------------------------------+-------------------------------------------+-----------------+
|Covered_Recipient_Profile_First_Name|Covered_Recipient_Profile_Last_Name|Covered_Recipient_Profile_Primary_Specialty|   total_payments|
+------------------------------------+-----------------------------------+-------------------------------------------+-----------------+
|                             STEPHEN|                           BURKHART|                       Allopathic & Oste...|    3.392202493E7|
|                             WILLIAM|                             BINDER|                       Allopathic & Oste...|    2.943435593E7|
|                               KEVIN|                              FOLEY|                       Allopathic & Oste...|    1.730653526E7|
|                                IVAN|                             OSORIO|                       Allopathic & Oste...|    1.606551551E7|
|                              GEORGE|   

The physician who received the highest payment was Dr. Stephen Burkhart with $33.9 million followed by Dr. William Binder with $29.4 million and Dr. Kevin Foley with $17.3 million among the top-ten highest-paid physicians who all specialized in "Allopathic & Osteopathic Physicians" and collectively received more than $131 million indicating their involvement with healthcare companies through intellectual property or consulting relationships or other valuable arrangements.